# 第八章 Data Wrangling: Join, Combine, 数据整理： 连接，合并

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Hierarchical Indexing 分层索引

In [3]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [ ]:
data.index

In [ ]:
data['b']
data['b':'c']
data.loc[['b', 'd']]

In [ ]:
data.loc[:, 2]

In [ ]:
data.unstack()

In [ ]:
data.unstack().stack()

In [21]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [24]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [8]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])

### Reordering and Sorting Levels

In [25]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [10]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [15]:
frame.sort_index(level=1) #[:1/0] key=1  key2 11，22
# frame.sort_index(level=0) #默认key0 aa, bb

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [16]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics by Level

In [26]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [27]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame's columns

In [28]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [31]:
frame2 = frame.set_index(['c', 'd'])
# frame2 = frame.set_index(['d', 'c'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [32]:
frame.set_index(['c', 'd'], drop=False) #保留原始字段

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [ ]:
frame2.reset_index()

## Combining and Merging Datasets

### Database-Style DataFrame Joins

In [37]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [36]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [38]:
pd.merge(df1, df2) # 只合并相同的行索引

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [41]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [59]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
df3,df4

(  lkey  data1
 0    b      0
 1    b      1
 2    a      2
 3    c      3
 4    a      4
 5    a      5
 6    b      6,
   rkey  data2
 0    a      0
 1    b      1
 2    d      2)

In [45]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')  #这里就是明确2个字段对应具体信息 类似于 mysql: select a.*, b.* from a inner join b on a.lkey=b.rkey 会多取一次合并字段
#合并的结果数量等于重复字段更多的DF的总数

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [48]:
pd.merge(df1, df2, how='outer')       # 自动合并key部分
# pd.merge(df2, df1, how='outer')   

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [57]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
df1,df2

(  key  data1
 0   b      0
 1   b      1
 2   a      2
 3   c      3
 4   a      4
 5   b      5,
   key  data2
 0   a      0
 1   b      1
 2   a      2
 3   b      3
 4   d      4)

In [51]:
pd.merge(df1, df2, on='key', how='left') #left outer join 笛卡尔积出现了 b:3*2 + a:2*2 + c:1 = 11

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [52]:
pd.merge(df1, df2, how='inner')  #inner join同样笛卡尔 b:3*2 + a:2*2 = 10

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [54]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
left, right

(  key1 key2  lval
 0  foo  one     1
 1  foo  two     2
 2  bar  one     3,
   key1 key2  rval
 0  foo  one     4
 1  foo  one     5
 2  bar  one     6
 3  bar  two     7)

In [55]:
pd.merge(left, right, on=['key1', 'key2'], how='outer') #独立的key1 + key2 出现了5次 foo one*2, foo two, bar one, bar two

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [60]:
print(pd.merge(left, right, on='key1')) # foo: 2*2 + bar: 2*1 = 6 因为key2字段相同，所以被自充重命名后缀_x, _y
pd.merge(left, right, on='key1', suffixes=('_left', '_right')) #后缀   

  key1 key2_x  lval key2_y  rval
0  foo    one     1    one     4
1  foo    one     1    one     5
2  foo    two     2    one     4
3  foo    two     2    one     5
4  bar    one     3    one     6
5  bar    one     3    two     7


,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on Index

In [3]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
print(left1)
right1

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5


,group_val
a,3.5
b,7.0


In [11]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [5]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [12]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
print(lefth)
righth

     key1  key2  data
0    Ohio  2000   0.0
1    Ohio  2001   1.0
2    Ohio  2002   2.0
3  Nevada  2001   3.0
4  Nevada  2002   4.0


event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [48]:
# pd.merge(lefth, righth, left_on=('key1', 'key2'), right_index = True) #元祖也是可行的 3+1*2
print("----内连接---")
print( lefth.join(righth, on=['key1', 'key2'], how='inner') ) #内连接
print("----左(外)连接---")
print( lefth.join(righth, on=['key1', 'key2']) )  # 默认左(外)连接
print("----全(外)连接---")
print( lefth.join(righth, on=['key1', 'key2'], how='outer') ) #全(外)连接
print("----内连接---")
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True) #默认内连接


#page253/541: In part for legacy reasons (i.e., much earlier versions of pandas), DataFrame’s join
# method performs a left join on the join keys, exactly preserving the left frame’s row
# index. It also supports joining the index of the passed DataFrame on one of the col‐
# umns of the calling DataFrame

----内连接---
     key1  key2  data  event1  event2
0    Ohio  2000   0.0       4       5
0    Ohio  2000   0.0       6       7
1    Ohio  2001   1.0       8       9
2    Ohio  2002   2.0      10      11
3  Nevada  2001   3.0       0       1
----左(外)连接---
     key1  key2  data  event1  event2
0    Ohio  2000   0.0     4.0     5.0
0    Ohio  2000   0.0     6.0     7.0
1    Ohio  2001   1.0     8.0     9.0
2    Ohio  2002   2.0    10.0    11.0
3  Nevada  2001   3.0     0.0     1.0
4  Nevada  2002   4.0     NaN     NaN
----全(外)连接---
     key1  key2  data  event1  event2
0    Ohio  2000   0.0     4.0     5.0
0    Ohio  2000   0.0     6.0     7.0
1    Ohio  2001   1.0     8.0     9.0
2    Ohio  2002   2.0    10.0    11.0
3  Nevada  2001   3.0     0.0     1.0
4  Nevada  2002   4.0     NaN     NaN
4  Nevada  2000   NaN     2.0     3.0
----内连接---


,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [40]:
# print( lefth.join(righth, on=['key1', 'key2'], how='outer') )
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer') #3 + 1*2 + [1 +1 ]out

     key1  key2  data  event1  event2
0    Ohio  2000   0.0     4.0     5.0
0    Ohio  2000   0.0     6.0     7.0
1    Ohio  2001   1.0     8.0     9.0
2    Ohio  2002   2.0    10.0    11.0
3  Nevada  2001   3.0     0.0     1.0
4  Nevada  2002   4.0     NaN     NaN
4  Nevada  2000   NaN     2.0     3.0


,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [22]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
print(left2)
right2

   Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0


,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [27]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [28]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [31]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [55]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
print(another,'\n', left2,'\n', right2)
print(left2.join([right2, another]))
left2.join([right2, another], how='outer')

   New York  Oregon
a       7.0     8.0
c       9.0    10.0
e      11.0    12.0
f      16.0    17.0 
    Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0 
    Missouri  Alabama
b       7.0      8.0
c       9.0     10.0
d      11.0     12.0
e      13.0     14.0
   Ohio  Nevada  Missouri  Alabama  New York  Oregon
a   1.0     2.0       NaN      NaN       7.0     8.0
c   3.0     4.0       9.0     10.0       9.0    10.0
e   5.0     6.0      13.0     14.0      11.0    12.0


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### Concatenating Along an Axis

In [60]:
arr = np.arange(12).reshape((3, 4))
print(arr)
print(np.concatenate([arr, arr], axis=0), np.concatenate([arr, arr], axis=0).shape) #axis 默认0: shape (2*3,4)
np.concatenate([arr, arr], axis=1), np.concatenate([arr, arr], axis=1).shape   #shape(3, 4*2)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]] (6, 4)


(array([[ 0,  1,  2,  3,  0,  1,  2,  3],
        [ 4,  5,  6,  7,  4,  5,  6,  7],
        [ 8,  9, 10, 11,  8,  9, 10, 11]]),
 (3, 8))

In [84]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
print( s1)
print( s2)
print( s3)

a    0
b    1
dtype: int64
c    2
d    3
e    4
dtype: int64
f    5
g    6
dtype: int64


In [62]:
pd.concat([s1, s2, s3]) # 一列

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [88]:
print(pd.concat([s1, s2, s3], axis=1))  #三列
pd.concat([s1, s2, s3], axis=1, keys=["hidden_column1","hidden_column2","hidden_column3"]) 

     0    1    2
a  0.0  NaN  NaN
b  1.0  NaN  NaN
c  NaN  2.0  NaN
d  NaN  3.0  NaN
e  NaN  4.0  NaN
f  NaN  NaN  5.0
g  NaN  NaN  6.0


,hidden_column1,hidden_column2,hidden_column3
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [70]:
s4 = pd.concat([s1, s3])
print(s4)
print(pd.concat([s1, s4], axis=1))
pd.concat([s1, s4], axis=1, join='inner')

a    0
b    1
f    5
g    6
dtype: int64
     0  1
a  0.0  0
b  1.0  1
f  NaN  5
g  NaN  6


,0,1
a,0,0
b,1,1


In [89]:
# ref https://stackoverflow.com/questions/61362942/concat-got-an-unexpected-keyword-argument-join-axes
# pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])   "join_axes" was deprecated in version 0.25 for some reason
print(pd.concat([s1, s4], axis=1))
pd.concat([s1, s4], axis=1).reindex(['a', 'c', 'b', 'e']) #保留ab去掉fg 增加ce[NA]

     0  1
a  0.0  0
b  1.0  1
f  NaN  5
g  NaN  6


,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


In [75]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
print(result)
result.unstack()

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64


,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [76]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [81]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
print(df1)
print(df2)
print(pd.concat([df1, df2]))
print(pd.concat([df1, df2],  keys=['level1', 'level2']))
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

   one  two
a    0    1
b    2    3
c    4    5
   three  four
a      5     6
c      7     8
   one  two  three  four
a  0.0  1.0    NaN   NaN
b  2.0  3.0    NaN   NaN
c  4.0  5.0    NaN   NaN
a  NaN  NaN    5.0   6.0
c  NaN  NaN    7.0   8.0
          one  two  three  four
level1 a  0.0  1.0    NaN   NaN
       b  2.0  3.0    NaN   NaN
       c  4.0  5.0    NaN   NaN
level2 a  NaN  NaN    5.0   6.0
       c  NaN  NaN    7.0   8.0


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [90]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [108]:
pd.concat([df1, df2]).index

Index(['a', 'b', 'c', 'a', 'c'], dtype='object')

In [113]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],names=['upper','lower']) #几级column就可以命名几个column name 由高到低
pd.concat([df1, df2], axis=1, keys=[('top_level1',('level1_1','level1_2')), ('top_level2',('level2_1','level2_2'))],names=['upper','lower', 'lowest'])
# pd.concat??

upper            top_level1               top_level2     
lower  (level1_1, level1_2)     (level2_1, level2_2)     
lowest                  one two                three four
a                         0   1                  5.0  6.0
b                         2   3                  NaN  NaN
c                         4   5                  7.0  8.0

In [114]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
print(df1)
df2

          a         b         c         d
0 -0.204708  0.478943 -0.519439 -0.555730
1  1.965781  1.393406  0.092908  0.281746
2  0.769023  1.246435  1.007189 -1.296221


,b,d,a
0,0.274992,0.228913,1.352917
1,0.886429,-2.001637,-0.371843


In [116]:
print(pd.concat([df1, df2]))#, ignore_index=True)
pd.concat([df1, df2], ignore_index=True) # 重组index

          a         b         c         d
0 -0.204708  0.478943 -0.519439 -0.555730
1  1.965781  1.393406  0.092908  0.281746
2  0.769023  1.246435  1.007189 -1.296221
0  1.352917  0.274992       NaN  0.228913
1 -0.371843  0.886429       NaN -2.001637


,a,b,c,d
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221
3,1.352917,0.274992,NaN,0.228913
4,-0.371843,0.886429,NaN,-2.001637


### Combining Data with Overlap

In [117]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
print(a)
print(b)
np.where(pd.isnull(a), b, a)

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64
f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64


In [ ]:
b[:-2].combine_first(a[2:])

In [ ]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
df1
df2
df1.combine_first(df2)

## Reshaping and Pivoting 重塑和透视图

### Reshaping with Hierarchical Indexing 分层索引重塑

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

In [ ]:
result = data.stack()
result

In [ ]:
result.unstack()

In [ ]:
result.unstack(0)
result.unstack('state')

In [ ]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2
data2.unstack()

In [ ]:
data2.unstack()
data2.unstack().stack()
data2.unstack().stack(dropna=False)

In [ ]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df
df.unstack('state')

In [ ]:
df.unstack('state').stack('side')

### Pivoting “Long” to “Wide” Format 长到宽？

In [53]:
data = pd.read_csv('examples/macrodata.csv')
# print(data.head())

periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
# print(periods)
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
# print(columns)
data = data.reindex(columns=columns)
# print(data)
# print( periods.to_timestamp('D', 'end'))
data.index = periods.to_timestamp('D', 'end')
print(data)
# print(data.stack())
ldata = data.stack().reset_index().rename(columns={0: 'value'})

item                             realgdp  infl  unemp
date                                                 
1959-03-31 23:59:59.999999999   2710.349  0.00    5.8
1959-06-30 23:59:59.999999999   2778.801  2.34    5.1
1959-09-30 23:59:59.999999999   2775.488  2.74    5.3
1959-12-31 23:59:59.999999999   2785.204  0.27    5.6
1960-03-31 23:59:59.999999999   2847.699  2.31    5.2
...                                  ...   ...    ...
2008-09-30 23:59:59.999999999  13324.600 -3.16    6.0
2008-12-31 23:59:59.999999999  13141.920 -8.79    6.9
2009-03-31 23:59:59.999999999  12925.410  0.94    8.1
2009-06-30 23:59:59.999999999  12901.504  3.37    9.2
2009-09-30 23:59:59.999999999  12990.341  3.56    9.6

[203 rows x 3 columns]


In [61]:
# data = pd.read_csv('examples/macrodata.csv')
# data
# data.reset_index()
# print(data.head(),data.shape,data.stack().shape)
print(data)
# data.stack().reset_index().rename(columns={0:'value'})

item                             realgdp  infl  unemp
date                                                 
1959-03-31 23:59:59.999999999   2710.349  0.00    5.8
1959-06-30 23:59:59.999999999   2778.801  2.34    5.1
1959-09-30 23:59:59.999999999   2775.488  2.74    5.3
1959-12-31 23:59:59.999999999   2785.204  0.27    5.6
1960-03-31 23:59:59.999999999   2847.699  2.31    5.2
...                                  ...   ...    ...
2008-09-30 23:59:59.999999999  13324.600 -3.16    6.0
2008-12-31 23:59:59.999999999  13141.920 -8.79    6.9
2009-03-31 23:59:59.999999999  12925.410  0.94    8.1
2009-06-30 23:59:59.999999999  12901.504  3.37    9.2
2009-09-30 23:59:59.999999999  12990.341  3.56    9.6

[203 rows x 3 columns]


In [42]:
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [52]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [46]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-0.204708
1,1959-03-31 23:59:59.999999999,infl,0.000,0.478943
2,1959-03-31 23:59:59.999999999,unemp,5.800,-0.519439
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-0.555730
4,1959-06-30 23:59:59.999999999,infl,2.340,1.965781
5,1959-06-30 23:59:59.999999999,unemp,5.100,1.393406
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.092908
7,1959-09-30 23:59:59.999999999,infl,2.740,0.281746
8,1959-09-30 23:59:59.999999999,unemp,5.300,0.769023
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,1.246435


In [47]:
pivoted = ldata.pivot('date', 'item')
print(pivoted[:5])
pivoted['value'][:5]

                              value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.478943 -0.204708   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  1.965781 -0.555730   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  0.281746  0.092908   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  1.007189  1.246435   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.228913  0.274992   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.519439  
1959-06-30 23:59:59.999999999  1.393406  
1959-09-30 23:59:59.999999999  0.769023  
1959-12-31 23:59:59.999999999 -1.296221  
1960-03-31 23:59:59.999999999  1.352917  


item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [60]:
# ldata??
print(ldata.set_index(['date', 'item']))
unstacked = ldata.set_index(['date', 'item']).unstack('item')
# print(unstacked.index, unstacked.columns)
unstacked[:7]

                                           value
date                          item              
1959-03-31 23:59:59.999999999 realgdp   2710.349
                              infl         0.000
                              unemp        5.800
1959-06-30 23:59:59.999999999 realgdp   2778.801
                              infl         2.340
...                                          ...
2009-06-30 23:59:59.999999999 infl         3.370
                              unemp        9.200
2009-09-30 23:59:59.999999999 realgdp  12990.341
                              infl         3.560
                              unemp        9.600

[609 rows x 1 columns]


value                
item                           infl   realgdp unemp
date                                               
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6

### Pivoting “Wide” to “Long” Format 

In [2]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [3]:
# print(pd.melt(df, ['key', 'A']))  #  (key,A)*(B,C)  3*2 = 6行
# print(pd.melt(df, ['key', 'A', 'B'])) # (key,A,B)*(C) 3*1 = 3行
melted = pd.melt(df, ['key'])     # 3*3    (Key)*(A,B,C) = 9行
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [4]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [6]:
print(reshaped.reset_index().index)
reshaped.reset_index()


RangeIndex(start=0, stop=3, step=1)


variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [9]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [10]:
print(pd.melt(df, value_vars=['A', 'B', 'C']))
pd.melt(df, value_vars=['key', 'A', 'B'])

  variable  value
0        A      1
1        A      2
2        A      3
3        B      4
4        B      5
5        B      6
6        C      7
7        C      8
8        C      9


,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6


## Conclusion